In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from apyori import apriori
#from efficient_apriori import apriori

In [2]:
data = pd.read_excel("../Datasets/HVAC.xlsx", "HISTORICO_DATOS")

In [3]:
variables_a_eliminar = ["C_O_P_ BOMBA CALOR FELIPE", "C_O_P_ BOMBA CALOR CARLOS",
                        "C_O_P_ INSTALACIÓN GRUPO FRÍO 1", "C_O_P_ INSTALACÍON GRUPO FRÍO 2",
                        "ORDEN", "VÁLVULA BY PASS SECUNDARIO FRÍO",
                        "TEMPERATURA CONTROL DE BY PASS SECUNDARIO", "SECUNDARIO FRÍO 1",
                        "SECUNDARIO FRÍO 2", "SECUNDARIO FRÍO 3", "MODO INVIERNO BC1", 
                        "MODO INVIERNO BC2", "PERIODO P6", "CONTROL CALOR", 
                        "CAPACIDAD BOMBA CALOR FELIPE %", "CAPACIDAD BOMBA CALOR CARLOS %",
                        "CAPACIDAD GRUPO DE FRÍO 1"," CAPACIDAD GRUPO DE FRÍO 2",
                        "IMPULSIÓN SECUNDARIO CALOR", "SECUNDARIO CALOR 1", 
                        "SECUNDARIO CALOR 2", "SECUNDARIO CALOR 3"]
lista_variables = data.columns.values.tolist()
subLista = [x for x in lista_variables if x not in variables_a_eliminar]
data = data[subLista]

data = data
data["POTENCIA TERMICA BOMBA CALOR CARLOS"] = data["KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS"] * 0.001163
data["POTENCIA TERMICA BOMBA CALOR FELIPE"] = data["KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE"] * 0.001163
data["POTENCIA TERMICA GRUPO FRIO 1"] = data["KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1"] * 0.001163
data["POTENCIA TERMICA GRUPO FRIO 2"] = data["KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2"] * 0.001163

data.head()

,Fecha- hora de lectura,POTENCIA TRAFO 2,POTENCIA TRAFO 3,POTENCIA TRAFO 4,POTENCIA TRAFO 5,POTENCIA MEDIA CONECTADA,CONTROL FRÍO,POTENCIA BOMBA CALOR FELIPE,POTENCIA BOMBA CALOR CARLOS,TEMPERATURA AMBIENTE BOMBA CALOR CARLOS,...,TEMPERATURA SALIDA BOMBA CALOR CARLOS,TEMPERATURA SALIDA BOMBA CALOR FELIPE,KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS,KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2,POTENCIA TERMICA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR FELIPE,POTENCIA TERMICA GRUPO FRIO 1,POTENCIA TERMICA GRUPO FRIO 2
0,2016-01-01 00:00:00,0.783603,38.338665,232.182602,235.755234,557.290039,26.984179,7.253334,7.061334,17.268667,...,36.190666,36.192665,28599.992188,26129.994141,4400.000488,29349.998047,33.261791,30.389183,5.117201,34.134048
1,2016-01-01 00:15:00,0.926969,38.338665,197.970230,113.384056,372.547760,26.805664,0.800000,0.480000,18.049334,...,33.625999,32.770664,163366.671875,193396.765625,18550.000000,0.000000,189.995439,224.920438,21.573650,0.000000
2,2016-01-01 00:30:00,0.793163,38.455196,211.676941,236.968918,469.299347,26.055836,0.800000,0.480000,17.046000,...,33.247997,32.559998,103306.765625,137973.359375,218650.000000,96750.000000,120.145768,160.463017,254.289950,112.520250
3,2016-01-01 00:45:00,0.860073,38.688263,206.066818,298.481995,570.994995,25.266325,0.800000,0.480000,13.052667,...,32.989998,32.559998,73883.296875,102829.898438,247150.000000,214850.000000,85.926274,119.591172,287.435450,249.870550
4,2016-01-01 01:00:00,0.812257,38.688263,193.984100,292.789215,563.804016,25.605207,0.800000,0.480000,10.656666,...,32.719997,32.242664,50396.769531,81683.398438,238550.000000,203300.000000,58.611443,94.997792,277.433650,236.437900


## Reglas de Asociación con el algoritmo Apriori

In [4]:
from efficient_apriori import apriori
transactions = [('eggs', 'bacon', 'soup'),
                ('eggs', 'bacon', 'apple'),
                ('soup', 'bacon', 'banana')]
itemsets, rules = apriori(transactions, min_support=0.2,  min_confidence=1)

# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{apple, eggs} -> {bacon} (conf: 1.000, supp: 0.333, lift: 1.000, conv: 0.000)
{banana, soup} -> {bacon} (conf: 1.000, supp: 0.333, lift: 1.000, conv: 0.000)
{eggs, soup} -> {bacon} (conf: 1.000, supp: 0.333, lift: 1.000, conv: 0.000)
{apple, bacon} -> {eggs} (conf: 1.000, supp: 0.333, lift: 1.500, conv: 333333333.333)
{bacon, banana} -> {soup} (conf: 1.000, supp: 0.333, lift: 1.500, conv: 333333333.333)


In [5]:
dataCarlos = data[['POTENCIA BOMBA CALOR CARLOS', 'POTENCIA TERMICA BOMBA CALOR CARLOS', 'TEMPERATURA EXTERIOR',
                   'TEMPERATURA SALIDA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS']]

In [6]:
dataCarlos.describe()

,POTENCIA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR CARLOS,TEMPERATURA EXTERIOR,TEMPERATURA SALIDA BOMBA CALOR CARLOS,C_O_P BOMBA CALOR CARLOS
count,79285.000000,79285.000000,79285.000000,79285.000000,79285.000000
mean,33.616413,132.154739,15.083257,29.447289,1.603675
std,43.907480,149.917857,11.345877,13.676912,2.059181
min,0.000000,0.000000,-1.400000,0.000000,0.000000
25%,0.480000,9.877769,6.968000,16.940001,0.000000
50%,0.500000,46.868723,13.353139,34.740002,0.000000
75%,67.599998,255.914298,23.717333,42.450001,3.689127
max,164.206665,592.614391,43.799999,47.889999,74.741554


In [7]:
dataCarlos["label_potencia"] = ' '
dataCarlos["label_potenciaTermica"] = ' '
dataCarlos["label_temperaturaExterior"] = ' '
dataCarlos["label_temperaturaSalida"] = ' '
dataCarlos["label_COP"] = ' '
dataCarlos.head()

C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dag17\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,POTENCIA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR CARLOS,TEMPERATURA EXTERIOR,TEMPERATURA SALIDA BOMBA CALOR CARLOS,C_O_P BOMBA CALOR CARLOS,label_potencia,label_potenciaTermica,label_temperaturaExterior,label_temperaturaSalida,label_COP
0,7.061334,33.261791,11.696458,36.190666,0.7453,,,,,
1,0.480000,189.995439,11.142512,33.625999,0.0000,,,,,
2,0.480000,120.145768,11.005099,33.247997,0.0000,,,,,
3,0.480000,85.926274,10.627213,32.989998,0.0000,,,,,
4,0.480000,58.611443,10.664143,32.719997,0.0000,,,,,


In [8]:
dataCarlosArr = dataCarlos.to_numpy()
size_dataCarlosArr = dataCarlos.shape
filas = size_dataCarlosArr[0]
columnas = size_dataCarlosArr[1]
for i in range(filas):
    if (float(dataCarlosArr[i][0]) > 43) and float((dataCarlosArr[i][0]) < 63):
        dataCarlosArr[i][5] = 'potencia buena'
    elif (float(dataCarlosArr[i][0]) <= 43):
        dataCarlosArr[i][5] = 'potencia baja'
    elif (float(dataCarlosArr[i][0]) >= 63):
        dataCarlosArr[i][5] = 'potencia alta'

In [9]:
for i in range(filas):
    if (float(dataCarlosArr[i][1]) > 132) and (float(dataCarlosArr[i][1]) < 255):
        dataCarlosArr[i][6] = 'potencia termica buena'
    elif (float(dataCarlosArr[i][1]) <= 132):
        dataCarlosArr[i][6] = 'potencia termica baja'
    elif (float(dataCarlosArr[i][1]) >= 255):
        dataCarlosArr[i][6] = 'potencia termica alta'

In [10]:
for i in range(filas):
    if (float(dataCarlosArr[i][2]) > 10) and (float(dataCarlosArr[i][2]) < 20):
        dataCarlosArr[i][7] = 'temp ext buena'
    elif (float(dataCarlosArr[i][2]) <= 10):
        dataCarlosArr[i][7] = 'temp ext baja'
    elif (float(dataCarlosArr[i][2]) >= 20):
        dataCarlosArr[i][7] = 'temp ext alta'

In [11]:
for i in range(filas):
    if (float(dataCarlosArr[i][3]) > 21) and (float(dataCarlosArr[i][3]) < 42):
        dataCarlosArr[i][8] = 'temp sal buena'
    elif (float(dataCarlosArr[i][3]) <= 21):
        dataCarlosArr[i][8] = 'temp sal baja'
    elif (float(dataCarlosArr[i][3]) >= 42):
        dataCarlosArr[i][8] = 'temp sal alta'

In [12]:
for i in range(filas):
    if (float(dataCarlosArr[i][4]) > 3) and (float(dataCarlosArr[i][4]) < 5):
        dataCarlosArr[i][9] = 'cop bueno'
    elif (float(dataCarlosArr[i][4]) <= 3):
        dataCarlosArr[i][9] = 'cop bajo'
    elif (float(dataCarlosArr[i][4]) >= 5):
        dataCarlosArr[i][9] = 'cop alto'

In [13]:
namesCol = dataCarlos.columns.values
dataCarlos_label = pd.DataFrame.from_records(dataCarlosArr, columns = namesCol)

In [14]:
dataCarlosApriori = dataCarlos_label[['label_potencia', 'label_potenciaTermica', 
                                      'label_temperaturaExterior', 'label_temperaturaSalida', 'label_COP']]
dataCarlosApriori

,label_potencia,label_potenciaTermica,label_temperaturaExterior,label_temperaturaSalida,label_COP
0,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
1,potencia baja,potencia termica buena,temp ext buena,temp sal buena,cop bajo
2,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
3,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
4,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
5,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
6,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
7,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
8,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo
9,potencia baja,potencia termica baja,temp ext buena,temp sal buena,cop bajo


In [15]:
def inspect(results):
    rh          = [tuple(result[2][0][0]) for result in results]
    lh          = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(rh, lh, supports, confidences, lifts))

In [16]:
dataCarlosNP = dataCarlosApriori.to_numpy()
transactionsCarlos = list(map(tuple, dataCarlosNP))

In [25]:
from apyori import apriori
rules = apriori(transactionsCarlos, min_support = 0.000002, min_confidence = 0.8, min_lift = 3, min_length = 2)
results = list(rules)
# Este comamdo crea un frame para ver los datos resultados
resultDataFrame=pd.DataFrame(inspect(results),
                columns=['rhs','lhs','support','confidence','lift'])

In [26]:
resultDataFrame

,rhs,lhs,support,confidence,lift
0,"(potencia alta,)","(potencia termica alta,)",0.244725,0.898703,3.574480
1,"(potencia buena,)","(potencia termica buena,)",0.065195,0.945664,7.193416
2,"(potencia alta, cop alto)","(potencia termica alta,)",0.000353,0.875000,3.480203
3,"(potencia termica alta, potencia baja)","(cop alto,)",0.002787,0.887550,86.983211
4,"(potencia buena, cop alto)","(potencia termica buena,)",0.000328,0.866667,6.592504
5,"(cop alto, temp sal baja)","(temp ext alta,)",0.007858,0.974961,3.019404
6,"(cop bajo, potencia termica alta)","(potencia alta,)",0.029388,0.986452,3.622550
7,"(cop bajo, potencia buena)","(potencia termica buena,)",0.001980,0.887006,6.747217
8,"(potencia alta, cop bueno)","(potencia termica alta,)",0.214984,0.958608,3.812742
9,"(potencia buena, cop bueno)","(potencia termica buena,)",0.062887,0.948089,7.211862
